# SLAM tutorial

SLAM and factor graphs tutorial prepared for the ORIentate Seminars

Author: Matias Mattamala (matias@robots.ox.ac.uk, [mmattamala@github](https://github.com/mmattamala))

Date: 23/11/2023


> ⚠️ **Warning**: Before proceeding, make sure you installed all the required dependencies in the [`requirements.txt`](../requirements.txt) file!


## Preliminaries

In [ ]:
!pip install -e ../

## Inspecting a SLAM graph

In [ ]:
import slam_tutorial

In [ ]:
from slam_tutorial.ncd_helpers import load_ground_truth
df = load_ground_truth(slam_tutorial.ASSETS_DIR + "/graphs/ground_truth.csv")

In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd

viz_clouds = []
x_tm1 = y_tm1 = z_tm1 = None
for s, ns, x_t, y_t, z_t, qx_t, qy_t, qz_t, qw_t in zip(df['#sec'], df['nsec'], df['x'], df['y'], df['z'], df['qx'], df['qy'], df['qz'], df['qw']):
    pose = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=np.array([x_t, y_t, z_t]))
    viz_clouds.append(pose)

In [ ]:
o3d.visualization.draw_geometries(
            viz_clouds[:60],
            window_name="ground_truth",
)

In [ ]:
filt_df